# Data Collection: มาเก็บรวบรวมข้อมูลจากแหล่งต่าง ๆ (DB & REST API) กันเถอะ!

# Get data from MySQL database

## Install PyMySQL 
ซึ่งเป็น package สำหรับเชื่อมต่อ MySQL database

In [24]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


## Config DB credential
ಥ_ಥ  คำเตือน: Cell ด้านล่างนี้เป็นความลับสุดยอด เขียนขึ้นมาเพื่อให้เห็นวิธีการในการเชื่อมต่อ database เท่านั้น **ห้ามเอา "plain text"แบบนี้ไปใช้ในชีวิตจริง**

มีข้อปฏิบัติในการเก็บรักษาไฟล์ที่เป็นความลับ (secret) ดังนี้
*   **ห้าม**เขียน credential (ความลับ) ลงมาใน code ตรง ๆ
*   **ห้าม** commit credential ลง Git เด็ดขาด
*   credential ควรเป็น environment variable / ไฟล์ .env / หรือ config file ที่เหมาะสม
*   **ห้าม** commit config file หรือ .env ไฟล์ดังกล่าวขึ้น Git ด้วย
*   ควรใช้ระบบ secret management เพื่อเก็บ credential อย่างปลอดภัย เช่น Vault หรือ Secret Manager
*   การขัดต่อคำแนะนำตามที่กล่าวมาถือว่าบาป



In [27]:
# TODO: ยอมเป็นคนบาปเพื่อการเรียนรู้
class Config:
  MYSQL_HOST = 'db4free.net'
  MYSQL_PORT = 3306  # default port ของ MySQL คือ 3306
  MYSQL_USER = 'datath'
  MYSQL_PASSWORD = 'DataScience-chillchill'
  MYSQL_DB = 'detraining'
  MYSQL_CHARSET = 'utf8mb4'

## Connect to DB
หลังจากที่มี Credential ของ database แล้วก็ connectโลด

In [31]:
import pymysql.cursors

# Connect to the database
connection = pymysql.connect(host=Config.MYSQL_HOST,
                             port=Config.MYSQL_PORT,
                             user=Config.MYSQL_USER,
                             password=Config.MYSQL_PASSWORD,
                             db=Config.MYSQL_DB,
                             charset=Config.MYSQL_CHARSET,
                             cursorclass=pymysql.cursors.DictCursor)

In [32]:
connection

ตัวแปร connection นี้ เราได้ connect ต่อเข้ากับ database เอาไว้แล้ว
## List Tables
เรามาลองดูกันว่ามี table อะไรในนั้นบ้าง

In [33]:
# list all tables
cursor = connection.cursor()
cursor.execute("show tables;")
tables = cursor.fetchall()
cursor.close()
print(tables)

[{'Tables_in_detraining': 'current_dept_emp'}, {'Tables_in_detraining': 'departments'}, {'Tables_in_detraining': 'dept_emp'}, {'Tables_in_detraining': 'dept_emp_latest_date'}, {'Tables_in_detraining': 'dept_manager'}, {'Tables_in_detraining': 'employees'}, {'Tables_in_detraining': 'online_retail'}, {'Tables_in_detraining': 'salaries'}, {'Tables_in_detraining': 'titles'}]


จากโค้ดตัวอย่างด้านบนจะเห็นได้ว่า การคิวรี่ database ทุกครั้ง เราจะต้องสร้าง `cursor` ขึ้นมาเพื่อ query SQL นั้น แล้วก็ปิด cursor ทุกครั้งหลังจบ 

(เหตุผลเป็นไปตามการออกแบบ database สามารถ[อ่านเกี่ยวกับ cursor เพิ่มเติมได้ที่นี่](https://en.wikipedia.org/wiki/Cursor_(databases)))

ดังนั้น จึงนิยมใช้คำสั่ง `with` ในการจัดการสร้าง cursor ขึ้นมา เมื่อจบคำสั่ง cursor จะถูก close ไปเองโดยอัตโนมัติเมื่อออกนอก scope ของ `with`

༼ つ ◕_◕ ༽つ *ขอความกรุณาไม่ drop database หรือลบ data ออก เพราะ database นี้ใช้กันหลายคน :-)*

## Query Table

In [34]:
# ใข้ with statement แทน cursor.close()
# TODO: มาลอง Query ข้อมูลจาก table online_retail ดูกัน

with connection.cursor() as cursor:
  # Read a single record
  sql = "SELECT * FROM online_retail"
  cursor.execute(sql)
  result = cursor.fetchall()

print("number of rows: ", len(result), "⁀⊙﹏☉⁀")

number of rows:  161331 ⁀⊙﹏☉⁀


Row เยอะแบบนี้ print ออกมาดูไม่ได้

ใช้งานลำบากอีก ขอแนะนำว่า `Pandas` ช่วยคุณได้ ʕ•́ᴥ•̀ʔ

## Convert to Pandas

In [35]:
import pandas as pd

In [37]:
retail = pd.DataFrame(result)

อ่าว เสร็จแล้วเหรอ ง่ายจัง ( ❛ ᴗ ❛ )

เราก็ได้ data ของเรามาแล้ว

In [38]:
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,1/12/2018 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,1/12/2018 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,1/12/2018 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,1/12/2018 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,1/12/2018 08:26,3.39,17850.0,United Kingdom


In [41]:
retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161331 entries, 0 to 161330
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    161331 non-null  int64  
 1   StockCode    161331 non-null  object 
 2   Description  160701 non-null  object 
 3   Quantity     161331 non-null  int64  
 4   InvoiceDate  161331 non-null  object 
 5   UnitPrice    161331 non-null  float64
 6   CustomerID   111388 non-null  float64
 7   Country      161331 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 9.8+ MB




---


# Get data from REST API

หลังจากต่อกับ Database แล้ว ก็มายิ่ง REST API กัน

Package `requests` ใช้สำหรับการยิง REST API

(โดยปกติต้อง install package นี้เพิ่มเติม แต่ colab pre-install ไว้อยู่แล้ว)

วิธีการ install: `pip install requests`

In [42]:
import requests

ลองคลิกดูผลลัพธ์ผ่าน web browser ได้ [Currency conversion API](https://de-training-2020-7au6fmnprq-de.a.run.app/currency_gbp/all) 

ผลลัพธ์ที่ return กลับมาจะเป็นประเภท JSON
จึงต้องใช้ package `json` (built-in) เพื่อโหลดข้อมูลเป็น dictionary

การที่เราสามารถยิง request และ output ออกมาได้เลยโดยที่ไม่ต้องสร้าง payload เพิ่ม ดังตัวอย่างนี้ เรียกว่า HTTP GET (ในกรณีอื่น ๆ สามารถเพิ่ม arguement หรือ query string เข้าไปใน URL ได้)

## Requests library
สามารถศึกษาวิธีการสร้าง request และการใช้งาน package `requests` [ได้ที่นี่](https://requests.readthedocs.io/en/master/)

In [45]:
url = "https://de-training-2020-7au6fmnprq-de.a.run.app/currency_gbp/all"
# TODO: ลองศึกษาวิธีการใช้งาน package requests จากลิ้งค์ด้านบน แล้วลองเขียนโค้ดเพื่อ call URL นี้

response = requests.get(url, auth=('user', 'pass'))

มาดูประเภทข้อมูลกัน

In [58]:
type(response.text)

str

## JSON loading
แต่ JSON ที่เราเห็นมันเป็นเพียงแค่ string

เราจะแปลง string ให้กลายเป็น dictionary!

เราเลยต้อง import `json` (built-in package)

In [47]:
import json

มาดูวิธีการใช้เลยดีกว่า สามารถกดดู help แบบนี้ได้ สำหรับ Jupyter notebook สะดวกสุด ๆ

In [48]:
? json

In [54]:
# TODO: ลองอ่าน string JSON ที่ได้รับ ให้กลายเป็น dictionary
result = response.json()

มาเช็คประเภทข้อมูล

In [57]:
type(result)

dict

In [61]:
result_conversion_rate = json.loads(response.text)
type(result_conversion_rate)

dict

In [62]:
print(type(result_conversion_rate))
assert isinstance(result_conversion_rate, dict)

<class 'dict'>


##สรุป ทำสองวิธี ให้ผลลัพธ์เหมือนกัน

 ## Convert to Pandas
 แปลงกันอีกครั้งหนึ่ง ʕ•́ᴥ•̀ʔ

In [63]:
conversion_rate = pd.DataFrame.from_dict(result_conversion_rate)

In [64]:
conversion_rate[:3]

,Rate
2018-01-01T00:00:00.000Z,43.9991
2018-01-02T00:00:00.000Z,44.0731
2018-01-03T00:00:00.000Z,43.7025


In [65]:
conversion_rate = conversion_rate.reset_index().rename(columns={"index":"date"})
conversion_rate[:3]

,date,Rate
0,2018-01-01T00:00:00.000Z,43.9991
1,2018-01-02T00:00:00.000Z,44.0731
2,2018-01-03T00:00:00.000Z,43.7025


In [66]:
conversion_rate[:3]

,date,Rate
0,2018-01-01T00:00:00.000Z,43.9991
1,2018-01-02T00:00:00.000Z,44.0731
2,2018-01-03T00:00:00.000Z,43.7025


ที่ต้องใช้ `.from_dict()` เพราะว่า ข้อมูลมี key ที่แตกต่างกันออกไป คือ index ที่เป็นเวลา เพราะฉะนั้น `.from_dict()` จึงเข้ามาช่วนการโหลดข้อมูลลักษณะแบบนี้

In [69]:
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceTimestamp
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,1/12/2018 08:26,2.55,17850.0,United Kingdom,1/12/2018 08:26
1,536365,71053,WHITE METAL LANTERN,6,1/12/2018 08:26,3.39,17850.0,United Kingdom,1/12/2018 08:26
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,1/12/2018 08:26,2.75,17850.0,United Kingdom,1/12/2018 08:26
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,1/12/2018 08:26,3.39,17850.0,United Kingdom,1/12/2018 08:26
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,1/12/2018 08:26,3.39,17850.0,United Kingdom,1/12/2018 08:26


# Join the data

ในตอนนี้เราจะนำข้อมูลการซื้อขายและข้อมูล Rate การแปลงค่าเงิน เราจะรวมข้อมูลจากทั้งสอง Dataframe มารวมกัน

เราจะนำข้อมูลจากทั้งสองมารวมกันผ่าน column InvoiceDate ใน retail และ date ใน conversion_rate 

แต่ถ้าสังเกตดี ๆ แล้วจะพบว่า InvoiceDate ใน retail จะเก็บข้อมูลในรูปแบบ datetime ส่วน date ใน conversion_rate จะเก็บข้อมูลในรูปแบบ timestamp ที่ส่วนเวลาเป็นเวลา 00:00:00 ทั้งหมด

In [68]:
# ก็อปปี้ column InvoiceDate เก็บเอาไว้ เผื่อได้ใช้ในอนาคต (อันใกล้) --> ไม่งั้นข้อมูล timestamp ของเราจะหายไป ಥ_ಥ
retail['InvoiceTimestamp'] = retail['InvoiceDate']

In [70]:
# แปลงให้ InvoiceDate ใน retail กับ date ใน conversion_rate มีเฉพาะส่วน date ก่อน
retail['InvoiceDate'] = pd.to_datetime(retail['InvoiceDate']).dt.date
conversion_rate['date'] = pd.to_datetime(conversion_rate['date']).dt.date

In [71]:
# รวม 2 dataframe เข้าด้วยกันด้วยคำสั่ง merge
final_df = retail.merge(conversion_rate, how="left", left_on="InvoiceDate", right_on="date")
final_df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceTimestamp,date,Rate
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2018-01-12,2.55,17850.0,United Kingdom,1/12/2018 08:26,2018-01-12,43.7705
1,536365,71053,WHITE METAL LANTERN,6,2018-01-12,3.39,17850.0,United Kingdom,1/12/2018 08:26,2018-01-12,43.7705
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2018-01-12,2.75,17850.0,United Kingdom,1/12/2018 08:26,2018-01-12,43.7705
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2018-01-12,3.39,17850.0,United Kingdom,1/12/2018 08:26,2018-01-12,43.7705
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2018-01-12,3.39,17850.0,United Kingdom,1/12/2018 08:26,2018-01-12,43.7705
...,...,...,...,...,...,...,...,...,...,...,...
161326,550638,22960,JAM MAKING SET WITH JARS,1,2019-04-19,8.29,NaN,United Kingdom,19/4/2019 15:44,2019-04-19,41.3622
161327,550638,22961,JAM MAKING SET PRINTED,1,2019-04-19,1.45,NaN,United Kingdom,19/4/2019 15:44,2019-04-19,41.3622
161328,550638,22978,PANTRY ROLLING PIN,1,2019-04-19,7.46,NaN,United Kingdom,19/4/2019 15:44,2019-04-19,41.3622
161329,550638,22982,PANTRY PASTRY BRUSH,1,2019-04-19,2.46,NaN,United Kingdom,19/4/2019 15:44,2019-04-19,41.3622


พอ join ข้อมูลได้แล้ว เราก็ มา คูณ currency conversion กัน (UnitPrice * Rate)

In [76]:
# TODO: เพิ่ม column 'THBPrice' ที่เกิดจาก column UnitPrice * Rate
# Hint: ลองดู apply function ของ pandas
def conversion_rate(price, unit):
    return price * unit
final_df['THBPrice'] = final_df.apply(lambda x: conversion_rate(x['UnitPrice'], x['Rate']), axis=1)
final_df.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceTimestamp,date,Rate,THBPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2018-01-12,2.55,17850.0,United Kingdom,1/12/2018 08:26,2018-01-12,43.7705,111.614775
1,536365,71053,WHITE METAL LANTERN,6,2018-01-12,3.39,17850.0,United Kingdom,1/12/2018 08:26,2018-01-12,43.7705,148.381995
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2018-01-12,2.75,17850.0,United Kingdom,1/12/2018 08:26,2018-01-12,43.7705,120.368875
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2018-01-12,3.39,17850.0,United Kingdom,1/12/2018 08:26,2018-01-12,43.7705,148.381995
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2018-01-12,3.39,17850.0,United Kingdom,1/12/2018 08:26,2018-01-12,43.7705,148.381995


## Save to CSV

In [77]:
# TODO: save "to csv" file
final_df.to_csv("output.csv", index=False)


<== กด ไอคอนรูป ไฟล์ ที่แถบด้านซ้ายเพื่อดูไฟล์ที่เซฟอยู่ใน directory :)

ลองกด Download มาดูได้

In [78]:
print("== End of Workshop 1 ʕ•́ᴥ•̀ʔっ♡ ==")

== End of Workshop 1 ʕ•́ᴥ•̀ʔっ♡ ==
